## Data Task

In [1]:
import pandas as pd
import re

def missing(data):
    return data.isnull().sum()

### I. Data Preparation (3 hours)

In [2]:
data_unstructured = pd.read_excel('Data.xlsx')
print("Missing Values by Column:")
missing(data_unstructured)

Missing Values by Column:


Student Index                                                                                   0
Round                                                                                           0
Masters Program                                                                                 1
Citizenship                                                                                     0
Sex                                                                                             0
Race                                                                                          194
Hispanic                                                                                      198
Age                                                                                             0
GPA                                                                                           142
College_Degree_Conferred_Date                                                                  79
Previous_Grad_School

In [3]:
data = data_unstructured.copy()

#1. YEARS SINCE GRADUATION
#assumption: the 79 people with missing degree conferral dates graduated at 21 years old
data["College_Degree_Conferred_Date"].fillna(data["Year of Survey"] - data["Age"] + 21, inplace = True)
data["College_Degree_Conferred_Date"] = [i[0:4] for i in data["College_Degree_Conferred_Date"].astype(str)]
data["Years_Since_Graduation"] = data["Year of Survey"] - data["College_Degree_Conferred_Date"].astype(int)

#2. WORK EXPERIENCE
#assumption: the 5 people with missing post-back work have no jobs/less than one year experience
data["Post-Bac_Work"] = data["Post-Bac_Work"].astype(str)
data["Post-Bac_Work"].fillna("Less than 1 year", inplace = True)
def work_experience(student):
    one_to_five = "(4-6|5-7)?\s?[0-5]{1}-?[0-5]?\s?(year|years)?"
    if student["Post-Bac_Work"] == "Less than 1 year":
        years = "< 1 year" 
    elif bool(re.match(one_to_five, student["Post-Bac_Work"])):
        years = "1 - 5 years" #includes 4-6, 5-7
    else:
        years = "Over 5 years" 
    return years
data["Work_Experience"] = data.apply(work_experience, axis = "columns")

#3. GPA
#data.groupby("Masters Program")["GPA"].mean()
adjusted_mean_gpa = round(data.loc[data["GPA"] <= 4.0, "GPA"].mean(), 2) 
data["GPA"].fillna(adjusted_mean_gpa, inplace = True)
data.loc[data["GPA"] > 4.0, "GPA"] = adjusted_mean_gpa

#4. RACE
def race(student):
    student["Race"] = str(student["Race"])
    if bool(re.match("^American Indian", student["Race"])):
        race = "American Indian/Alaska Native"
    elif bool(re.match("^Arab", student["Race"])):
        race = "Arab/Middle Eastern"
    elif bool(re.match("^Asian", student["Race"])):
        race = "Asian"
    elif bool(re.match("^Black", student["Race"])):
        race = "Black"
    elif bool(re.match("^White", student["Race"])): 
        race = "White"
    else: #"I prefer not to respond," "Other," and "Missing"
        race = "No Race Specified"
    return race
data["Race"] = data.apply(race, axis = "columns")

#5. OVERALL SATISFACTION
data["Overall Satisfaction"] = [i.lower() for i in data["Overall Satisfaction"]]
satisfaction_labels = ["very dissatisfied", "dissatisfied", 
                       "neutral", "satisfied", "very satisfied"]
data["Overall Satisfaction"].replace(satisfaction_labels, 
                                     [1, 2, 3, 4, 5], inplace = True)

#6. REMOVAL OF UNNECESSARY COLUMNS and TRIMMING OF COLUMN NAMES:
irrelevant_columns = ["Round",
                      "Post-Bac_Work",
                      "College_Degree_Conferred_Date"]
data.drop(irrelevant_columns, axis = "columns", inplace = True)

column_names = {
    "Please rate how satisfied you are with Harris performance in Area A": "satisfaction_A",
    "Please rate how satisfied you are with Harris performance in Area B": "satisfaction_B",
    "Please rate how satisfied you are with Harris performance in Area C": "satisfaction_C",
    "Which policy areas are you most interested in? (Please choose no more than 3 policy areas)": "policy_areas",
    "How much do you agree or disagree with the Statement A": "agree_statement_A",
    "How much do you agree or disagree with the Statement B": "agree_statement_B",
    "Overall Satisfaction": "satisfaction",
    "Year of Survey": "year"
}
data.rename(columns = column_names, inplace = True)
data.reset_index(inplace = True)

**There exist 121 students whose GRE verbal and quantitative percentile scores are both missing:**

In [4]:
#7. GRE
missing_gre_verbal = set(data.loc[data["GRE_Verbal_Percentile"].isnull(), "Student Index"])
missing_gre_quant = set(data.loc[data["GRE_Quantitative_Percentile"].isnull(), "Student Index"])
missing_gre_verbal == missing_gre_quant

True

**I will impute the missing GRE scores by the average verbal and quantitative scores in each Master's Program:**

In [5]:
gre_labels = ["GRE_Verbal_Percentile", "GRE_Quantitative_Percentile"]
gre_by_program = data.groupby("Masters Program")[gre_labels].mean().round(0).reset_index()
gre_by_program

,Masters Program,GRE_Verbal_Percentile,GRE_Quantitative_Percentile
0,Master of Arts (MA) Program,93.0,96.0
1,Master of Arts in International Development an...,59.0,80.0
2,Master of Arts in International Development an...,78.0,64.0
3,Master of Arts in Public Policy (MA),82.0,73.0
4,Master of Arts in Public Policy with Certifica...,76.0,89.0
5,Master of Public Policy (MPP),75.0,73.0
6,Master of Public Policy (MPP) Studies Program,80.0,76.0
7,Master of Science in Computational Analysis an...,87.0,84.0
8,Master of Science in Environmental Science and...,70.0,75.0


In [6]:
program_gre_dict = {}
for i in range(len(gre_by_program)):
    program_gre_dict[gre_by_program.iloc[i, 0]] = [j for j in gre_by_program.iloc[i, 1:3]]
program_gre_dict

{'Master of Arts (MA) Program': [93.0, 96.0],
 'Master of Arts in International Development and Policy': [59.0, 80.0],
 'Master of Arts in International Development and Policy (MAIDP)': [78.0,
  64.0],
 'Master of Arts in Public Policy (MA)': [82.0, 73.0],
 'Master of Arts in Public Policy with Certificate in Research Methods (MACRM)': [76.0,
  89.0],
 'Master of Public Policy (MPP)': [75.0, 73.0],
 'Master of Public Policy (MPP) Studies Program': [80.0, 76.0],
 'Master of Science in Computational Analysis and Public Policy (MSCAPP)': [87.0,
  84.0],
 'Master of Science in Environmental Science and Policy (MSESP)': [70.0, 75.0]}

In [7]:
for i in data.loc[data["Student Index"].isin(list(missing_gre_verbal)), "index"]:
    program = data.loc[i, "Masters Program"]
    if program in program_gre_dict.keys():
        data.loc[i, "GRE_Verbal_Percentile"] = program_gre_dict[program][0]
        data.loc[i, "GRE_Quantitative_Percentile"] = program_gre_dict[program][1]

**Lastly, a simple mode imputation will be performed to the statement and area satisfaction columns due to the low number of missing values:**

In [8]:
#8. areas A, B, and C; statements A and B
areas_statements_labels = ["agree_statement_A", "agree_statement_B",
                           "satisfaction_A", "satisfaction_B", "satisfaction_C"]
for i in areas_statements_labels:
    data[i].fillna(data[i].mode()[0], inplace = True)
    
satisfaction_by_area = ["satisfaction_A", "satisfaction_B", "satisfaction_C"]
agree_statements = ["agree_statement_A", "agree_statement_B"]

satisfaction_by_area_labels = {"Very Dissatisfied": 1,
                               "Dissatisfied": 2,
                               "Neutral": 3,
                               "Satisfied": 4,
                               "Very Satisfied": 5}
agree_statements_labels = {"Strongly Disagree": 1,
                           "Disagree": 2,
                           "Neutral": 3,
                           "Agree": 4,
                           "Strongly Agree": 5}

for i in satisfaction_by_area + agree_statements:
    if i in satisfaction_by_area:
        data[i].replace(satisfaction_by_area_labels, inplace = True)
    else:
        data[i].replace(agree_statements_labels, inplace = True)

In [9]:
print("Missing Values by Column After Data Cleaning")
missing(data)

Missing Values by Column After Data Cleaning


index                            0
Student Index                    0
Masters Program                  1
Citizenship                      0
Sex                              0
Race                             0
Hispanic                       198
Age                              0
GPA                              0
Previous_Grad_School_Degree    785
Area_of_Focus                   52
GRE_Verbal_Percentile            0
GRE_Quantitative_Percentile      0
year                             0
agree_statement_A                0
agree_statement_B                0
satisfaction_A                   0
satisfaction_B                   0
satisfaction_C                   0
policy_areas                   397
satisfaction                     0
Years_Since_Graduation           0
Work_Experience                  0
dtype: int64

In [10]:
data.head()

,index,Student Index,Masters Program,Citizenship,Sex,Race,Hispanic,Age,GPA,Previous_Grad_School_Degree,...,year,agree_statement_A,agree_statement_B,satisfaction_A,satisfaction_B,satisfaction_C,policy_areas,satisfaction,Years_Since_Graduation,Work_Experience
0,0,1,Master of Public Policy (MPP),Indonesia,F,Asian,No,39,3.56,NaN,...,2020,4,4,4,4,4,"Leadership and Management,Methods of Policy An...",4,14,1 - 5 years
1,1,2,Master of Public Policy (MPP),United States,M,White,No,29,3.83,NaN,...,2020,3,3,3,3,3,"Data Science,Economic and Fiscal Policy,Social...",3,8,Over 5 years
2,2,3,Master of Public Policy (MPP),Pakistan,F,Asian,No,25,3.74,NaN,...,2020,4,4,4,4,4,"Education Policy,Methods of Policy Analysis,So...",4,3,1 - 5 years
3,3,4,Master of Public Policy (MPP),United States,F,Black,No,22,3.67,NaN,...,2020,4,4,3,4,5,"Data Science,Economic and Fiscal Policy,Financ...",4,1,< 1 year
4,4,5,Master of Public Policy (MPP),United States,M,White,No,22,3.58,NaN,...,2020,4,4,5,4,4,"Environmental Policy,Global Security and Confl...",5,1,< 1 year


In [11]:
data.to_excel("data_cleaned.xlsx", index = False)